In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)  # reproducibility


idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
y_data = [1, 0, 2, 3, 3, 4]    # ihello

# As we have one batch of samples, we will change them to variables only once
inputs = Variable(torch.LongTensor(x_data))
labels = Variable(torch.LongTensor(y_data))

num_classes = 5
input_size = 5
embedding_size = 10  # embedding size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
num_layers = 1  # one-layer rnn


class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(input_size=embedding_size,
                          hidden_size=5, batch_first=True)
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        # Variables x -> embedding -> rnn -> Linear
        #                   hidden -> rnn
        emb = self.embedding(x)
        emb = emb.view(batch_size, sequence_length, -1)

        # Propagate embedding through RNN
        # Input: (batch, seq_len, embedding_size)
        # h_0: (num_layers * num_directions, batch, hidden_size)
        out, _ = self.rnn(emb, h_0)
        return self.fc(out.view(-1, num_classes))


# Instantiate RNN model
model = Model()
print(model)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Train the model
for epoch in range(100):
    outputs = model(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.data[0]))
    print("Predicted string: ", ''.join(result_str))

print("Learning finished!")

Model(
  (embedding): Embedding(5, 10)
  (rnn): RNN(10, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)
epoch: 1, loss: 1.700
Predicted string:  eleoii
epoch: 2, loss: 1.357
Predicted string:  eliolo
epoch: 3, loss: 1.081
Predicted string:  ihillo
epoch: 4, loss: 0.924
Predicted string:  ihihll
epoch: 5, loss: 0.831
Predicted string:  ihehll
epoch: 6, loss: 0.750
Predicted string:  ehelll
epoch: 7, loss: 0.682
Predicted string:  ehelll
epoch: 8, loss: 0.649
Predicted string:  ehelll
epoch: 9, loss: 0.618
Predicted string:  ehelll
epoch: 10, loss: 0.594
Predicted string:  ihelll
epoch: 11, loss: 0.567
Predicted string:  ihilll
epoch: 12, loss: 0.540
Predicted string:  ihilll
epoch: 13, loss: 0.508
Predicted string:  ihilll
epoch: 14, loss: 0.463
Predicted string:  ihelll
epoch: 15, loss: 0.433
Predicted string:  ihelll
epoch: 16, loss: 0.397
Predicted string:  ihelll
epoch: 17, loss: 0.366
Predicted string:  ihelll
epoch: 18, loss: 0.343
Predicted string: